<a href="https://colab.research.google.com/github/carlosperz88/dasany/blob/main/Creador_de_Mockups_Simple_(Superposici%C3%B3n_de_Im%C3%A1genes).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creador de Mockups Simple con Gradio en Colab

Este cuaderno te permite superponer una imagen de diseño (primer plano) sobre una imagen de plantilla (fondo), como crear un mockup de camiseta.

**Instrucciones:**
1.  **Instalar Librerías:** Ejecuta la primera celda de código para instalar `Pillow` y `Gradio`.
2.  **Iniciar Interfaz:** Ejecuta la segunda celda de código. Haz clic en el enlace público (`Running on public URL: ...`) que aparecerá.
3.  **Usar la Interfaz:**
    *   Sube tu imagen de **fondo** (ej: la foto de la camiseta).
    *   Sube tu imagen de **diseño** (ej: tu logo o gráfico, **preferiblemente PNG con transparencia**).
    *   Ajusta los deslizadores para cambiar la **posición (X, Y)** y la **escala** del diseño.
    *   La imagen resultante se actualizará automáticamente.
4.  **Descargar:** Haz clic derecho en la imagen de salida y selecciona "Guardar imagen como...".

**Nota Importante:** Para obtener los mejores resultados, la imagen de "Diseño" debe tener un fondo transparente (formato PNG). Si tu diseño tiene un fondo sólido que quieres eliminar, puedes usar primero el cuaderno de "Removedor de Fondo".

In [ ]:
print("Instalando Pillow y Gradio...")
# Pillow ya suele estar instalado en Colab, pero lo incluimos por si acaso.
!pip install Pillow gradio -q

print("Instalación completada.")

Instalando Pillow y Gradio...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
Instalación completada.


In [ ]:
import gradio as gr
from PIL import Image
import io
import math # Para cálculos de redimensionamiento

print("Importaciones listas.")

def create_mockup(background_img_pil, foreground_img_pil, pos_x, pos_y, scale):
    """
    Superpone una imagen de primer plano sobre una de fondo.

    Args:
        background_img_pil (PIL.Image.Image): Imagen de fondo.
        foreground_img_pil (PIL.Image.Image): Imagen de primer plano (diseño).
        pos_x (int): Coordenada X (horizontal) de la esquina superior izquierda del diseño.
        pos_y (int): Coordenada Y (vertical) de la esquina superior izquierda del diseño.
        scale (float): Factor de escala para redimensionar el diseño (1.0 = tamaño original).

    Returns:
        PIL.Image.Image: Imagen combinada.
    """
    if background_img_pil is None or foreground_img_pil is None:
        # Si falta alguna imagen, devuelve la que esté disponible o nada
        return background_img_pil if background_img_pil else None

    # Asegúrate de que la imagen de fondo esté en modo RGBA para poder pegar transparencia
    background = background_img_pil.convert("RGBA")
    foreground = foreground_img_pil.convert("RGBA") # Asegurar RGBA también en el diseño

    # 1. Redimensionar el diseño (foreground) según el factor de escala
    original_width, original_height = foreground.size
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)

    # Evitar tamaño cero si la escala es muy pequeña
    if new_width <= 0 or new_height <= 0:
        return background # Devolver el fondo original si la escala es inválida

    foreground_resized = foreground.resize((new_width, new_height), Image.Resampling.LANCZOS)

    # 2. Crear una copia del fondo para no modificar el original directamente
    #    en caso de que Gradio lo reutilice.
    combined_image = background.copy()

    # 3. Pegar el diseño redimensionado sobre la copia del fondo
    #    La posición (pos_x, pos_y) es la esquina superior izquierda donde se pega.
    #    El tercer argumento (foreground_resized) actúa como máscara si tiene canal alfa,
    #    lo que permite que las partes transparentes del PNG no tapen el fondo.
    try:
        combined_image.paste(foreground_resized, (pos_x, pos_y), foreground_resized)
    except Exception as e:
        print(f"Error al pegar la imagen: {e}")
        # Podría ocurrir si las coordenadas están muy fuera de los límites,
        # aunque paste suele manejarlo recortando. Devolvemos el fondo por si acaso.
        return background

    return combined_image

# --- Crear la Interfaz de Gradio ---
print("Creando la interfaz de Gradio...")

with gr.Blocks() as demo:
    gr.Markdown("# Creador de Mockups Simple")
    gr.Markdown("Sube un fondo y un diseño, luego ajusta posición y escala.")

    with gr.Row():
        with gr.Column():
            bg_input = gr.Image(type="pil", label="1. Imagen de Fondo (Plantilla)")
            fg_input = gr.Image(type="pil", label="2. Imagen de Diseño (Preferible PNG transparente)")
        with gr.Column():
            output_image = gr.Image(type="pil", label="Resultado (Mockup)")

    with gr.Row():
        pos_x_slider = gr.Slider(minimum=-200, maximum=2000, step=1, value=50, label="Posición X (Horizontal)")
        pos_y_slider = gr.Slider(minimum=-200, maximum=2000, step=1, value=50, label="Posición Y (Vertical)")
        scale_slider = gr.Slider(minimum=0.1, maximum=3.0, step=0.05, value=0.5, label="Escala del Diseño")

    # Conectar los inputs/controles a la función y al output
    inputs_list = [bg_input, fg_input, pos_x_slider, pos_y_slider, scale_slider]
    bg_input.change(create_mockup, inputs=inputs_list, outputs=output_image)
    fg_input.change(create_mockup, inputs=inputs_list, outputs=output_image)
    pos_x_slider.change(create_mockup, inputs=inputs_list, outputs=output_image)
    pos_y_slider.change(create_mockup, inputs=inputs_list, outputs=output_image)
    scale_slider.change(create_mockup, inputs=inputs_list, outputs=output_image)

    gr.Markdown("---")
    gr.Markdown("**Consejo:** Para la imagen de diseño, usa un archivo PNG con fondo transparente. Si no lo tienes, usa primero el 'Removedor de Fondo'.")
    gr.Markdown("Haz clic derecho en la imagen de 'Resultado' para guardarla.")


# Lanzar la interfaz
print("Lanzando la interfaz de Gradio...")
print("Cuando aparezca 'Running on public URL:', haz clic en ese enlace.")

demo.launch(share=True, debug=True)

print("\nLa interfaz debería estar corriendo. Si no ves un enlace público, revisa los mensajes de error.")
print("Para detener la interfaz, interrumpe la ejecución de esta celda (botón de stop).")

Importaciones listas.
Creando la interfaz de Gradio...
Lanzando la interfaz de Gradio...
Cuando aparezca 'Running on public URL:', haz clic en ese enlace.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0ee6a5d3a4f3610a4b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0ee6a5d3a4f3610a4b.gradio.live

La interfaz debería estar corriendo. Si no ves un enlace público, revisa los mensajes de error.
Para detener la interfaz, interrumpe la ejecución de esta celda (botón de stop).
